# LLM Zoomcamp - Week 1

## Running Elastic

In [2]:
!curl localhost:9200

{
  "name" : "2e59a01cd5c5",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "f_3EpyzfTfeoFXURNihcqA",
  "version" : {
    "number" : "8.18.1",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "df116ec6455476a07daafc3ded80e2bb1a3385ed",
    "build_date" : "2025-04-30T10:07:44.026929518Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.1",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [34]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [9]:
from tqdm.auto import tqdm

## ElasticSearch Indexing

In [35]:
from elasticsearch import Elasticsearch

In [36]:
es_client = Elasticsearch('http://localhost:9200') 

In [70]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.delete(index=index_name)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [71]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [110]:
query = 'I just disovered the course. Can I still join it?'

In [109]:
def elastic_search(query, course="data-engineering-zoomcamp", fields=None, size=5):
    fields = fields or ["question^3", "text"]
    print (fields, size)
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": fields,
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        print (hit['_score'])
        result_docs.append(hit['_source'])
    
    return result_docs

In [111]:
answer = elastic_search(query)

['question^3', 'text'] 5
79.96579
72.849266
54.47627
54.057133
45.245464


## Searching

In [113]:
query = "How do execute a command on a Kubernetes pod?"

In [114]:
answer = elastic_search(query, fields=["question^4", "text"])

['question^4', 'text'] 5
44.50556
35.433445
33.70974
33.2635
32.589073


## Filtering

In [96]:
query = "How do copy a file to a Docker container?"

In [97]:
elastic_search(query,  course="machine-learning-zoomcamp",size=3, fields=["question^3", "text", "section"])

['question^3', 'text', 'section'] 3
55.040073
50.016533
44.859562


[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

## Building a Prompt

In [84]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [85]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [98]:
answers = elastic_search(query,  course="machine-learning-zoomcamp",size=3, fields=["question^3", "text", "section"])

['question^3', 'text', 'section'] 3
55.040073
50.016533
44.859562


In [115]:
context = []
for answer in answers:
    context.append(context_template.format(text=answer['text'], question=answer['question']))

context = "\n\n".join(context)

In [116]:
context

'Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\nA: You can copy files from your local machine into a Docker container using the docker cp comman

In [117]:
question = "How do copy a file to a Docker container?"

In [118]:
prompt = prompt_template.format(question=question, context=context)

In [119]:
len(prompt)

1446

## Tokens

In [104]:
import tiktoken

In [105]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [120]:
o = encoding.encode(prompt)

In [121]:
len(o)

320